# LangGraph with Stock Data Analysis: Task

This notebook demonstrates how to use `langgraph` and `langchain` to create a workflow for retrieving and analyzing stock data using the `yfinance` library and OpenAI's GPT-3.5-turbo model.



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1338NmNEx2PhD_W-yFaBBsAVQD-Vt9gdX?usp=sharing)

## Installation

First, we need to install the necessary packages:

In [10]:
!pip install -q langgraph langchain langchain_openai langchainhub yfinance

In [11]:
!pip freeze | grep "lang\|openai\|yfinance"

google-ai-generativelanguage==0.6.4
google-cloud-language==2.13.3
langchain==0.2.1
langchain-core==0.2.2
langchain-openai==0.1.8
langchain-text-splitters==0.2.0
langchainhub==0.1.17
langcodes==3.4.0
langgraph==0.0.57
langsmith==0.1.63
language_data==1.2.0
libclang==18.1.1
openai==1.30.5
yfinance==0.2.40


In [12]:
from google.colab import userdata
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Task

Your task will be to create a `langchain`/`langgraph` application to answer the following questions:

- What are the current prices of X, Y, Z
- What was the highest closing price of X stock in the last 4 weeks?
- How many stock splits Y had in the last 10 years?

Bonus:
- Draw a plot to compary the prices for X and Z in the last 3 months

## Tool Definitions
We define tools for retrieving current stock information, historical data, and corporate actions using the `yfinance` library.

In [13]:
# Documentation
# https://pypi.org/project/yfinance/

In [14]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage
import yfinance as yf

@tool("TOOL_NAME")
def get_data():
    """
    ...
    """
    return

## Creating the Agent
We create an agent using `langchain` to handle stock-related queries.

In [15]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Get the prompt to use (default)
prompt = hub.pull("hwchase17/openai-functions-agent")

tools = []
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

## Defining the Workflow
We define the workflow using langgraph.

### Define the Agent
We define the agent and the function to execute tools.

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

# Define the agent
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define the function to execute tools
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    tool_to_use = {t.name: t for t in tools}[agent_action.tool]
    observation = tool_to_use.invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"

### Creating the Workflow Graph
We create the workflow graph and compile it into a LangChain Runnable.

In [17]:
from langgraph.graph import END, Graph

workflow = Graph()

### Define the workflow








In [18]:
'''
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
'''

'\nfrom IPython.display import Image, display\ndisplay(Image(graph.get_graph(xray=True).draw_mermaid_png()))\n'

## Running the Workflow
We run the workflow with sample queries and compare the results with baseline answers.

In [19]:
query = "What is the latest stock price for AAPL, MSFT, and GOOG?"

In [20]:
# baseline answer
companies = 'msft aapl goog'
tickers = yf.Tickers(companies)
for x in companies.split(" "):
    print(x, tickers.tickers[x.upper()].info['currentPrice'])

msft 429.17
aapl 190.29
goog 177.4


In [21]:
query = "What was the highest closing price of Apple stock in the last 4 weeks?"

In [22]:
# baseline answer
tickers.tickers['AAPL'].history(period="3mo").iloc[-35:].sort_values(['Close'], ascending=False).head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-05-21 00:00:00-04:00,191.089996,192.729996,190.919998,192.350006,42309400,0.0,0.0
2024-05-20 00:00:00-04:00,189.330002,191.919998,189.009995,191.039993,44361300,0.0,0.0
2024-05-22 00:00:00-04:00,192.270004,192.820007,190.270004,190.899994,34648500,0.0,0.0


In [23]:
query = "How many stock splits google had in the last 10 years?"

In [24]:
# baseline answer
tickers.tickers['GOOG'].actions

,Dividends,Stock Splits
Date,,
2014-03-27 00:00:00-04:00,0.0,2.002000
2015-04-27 00:00:00-04:00,0.0,1.002746
2022-07-18 00:00:00-04:00,0.0,20.000000
